Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [13]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

In [14]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [15]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [16]:
# windowing function

def create_windows_np(data, window_size, horizon, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy.

    Parameters:
    data (np.ndarray): Time series data with one dimension.
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict.
    shuffle (bool): Shuffle the windows or not.

    Returns:
    tuple: A tuple containing the input-output pairs (windows, targets) as NumPy arrays.
    """

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y



In [17]:
past, future = (5, 2)

X_train, y_train = create_windows_np(train_data,
                                  window_size=past,
                                  horizon=2,
                                  shuffle=False)

X_test, y_test = create_windows_np(test_data,
                                  window_size=past,
                                  horizon=1,
                                  shuffle=False)

# Reshape X_train and X_test to be 3D arrays for the GRU layer
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [18]:
inputs = keras.layers.Input(shape=(past,future))
x = layers.GRU(units=32, return_sequences=True)(inputs)
x = layers.GRU(units=32)(x)
outputs = layers.Dense(units=1)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5, 2)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 5, 32)          │         3,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,825 (38.38 KB)

 Trainable params: 9,825 (38.38 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 108.6313 - mae: 9.4820 - val_loss: 56.8524 - val_mae: 6.5386
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 35.9409 - mae: 4.9128 - val_loss: 33.1453 - val_mae: 4.7057
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 23.5939 - mae: 3.8605 - val_loss: 25.6289 - val_mae: 4.0619
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 20.9303 - mae: 3.6320 - val_loss: 21.5386 - val_mae: 3.7092
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 18.9731 - mae: 3.4674 - val_loss: 19.1823 - val_mae: 3.5139
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 17.3060 - mae: 3.3098 - val_loss: 17.9949 - val_mae: 3.4059
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 16.2029 - mae: 3.2070 - val_loss: 16.4357 - val_mae: 3.1904
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 13.9551 - mae: 2.9070 - val_loss: 14.1794 - val_mae: 2.9444
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms

In [20]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.4155 - mae: 1.8373
Test Loss: [5.552616119384766, 1.8376516103744507]


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


Decidi normalizar los datos porque sino el modelo me daba incluso algo peor aunque tenia una variable mas. Escalando me dio muchisimo mejor desempeño.

In [27]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

scaler = MinMaxScaler()
df_multi_scaled = scaler.fit_transform(df_multi)

## train - test split
train_data = df_multi_scaled[:3000].copy()
test_data = df_multi_scaled[3000:, :].copy()

Voy a redefinir la funcion de creacion de ventana para que pueda trabajar con mas
de una variable predictora. Lo mejor seria sobrescribir la funcion create_windows_np
pero para que las respuestas queden ordenadas voy a definir una nueva fucion create_windows_np_multi

In [30]:
## Create windows
def create_windows_np_multi(data, window_size, horizon, target_column_index, shuffle=False):
    """
    Creates a dataset from the given time series data using NumPy with multiple features.

    Parameters:
    data (np.ndarray): Time series data with multiple dimensions (features).
    window_size (int): The number of past time steps to use as input features.
    horizon (int): The number of future time steps to predict for the target.
    target_column_index (int): The index of the column in 'data' that is the target variable.
    shuffle (bool): Shuffle the windows or not.

    Returns:
    tuple: A tuple containing the input-output pairs (windows, targets) as NumPy arrays.
    """

    X, y = [], []
    n_features = data.shape[1] # Obtiene el número de features

    for i in range(len(data) - window_size - horizon + 1):
        # X ahora contiene las ventanas de todas las features
        X.append(data[i:i+window_size, :])
        # y sigue siendo el valor futuro de la columna objetivo
        y.append(data[i+window_size+horizon-1, target_column_index])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y

target_column_index = df_multi.columns.get_loc('Temp')

X_train, y_train = create_windows_np_multi(train_data,
                                           window_size=past,
                                           horizon=future,
                                           target_column_index=target_column_index,
                                           shuffle=False)

X_test, y_test = create_windows_np_multi(test_data,
                                         window_size=past,
                                         horizon=future,
                                         target_column_index=target_column_index,
                                         shuffle=False)

In [31]:
n_features = X_train.shape[2]

inputs = keras.layers.Input(shape=(past, n_features))

x = layers.GRU(units=32, return_sequences=True)(inputs)
x = layers.GRU(units=32)(x)
outputs = layers.Dense(units=1)(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 5, 2)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (None, 5, 32)          │         3,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,825 (38.38 KB)

 Trainable params: 9,825 (38.38 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 0.1226 - mae: 0.2818 - val_loss: 0.0200 - val_mae: 0.1110
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0153 - mae: 0.0969 - val_loss: 0.0141 - val_mae: 0.0906
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0145 - mae: 0.0951 - val_loss: 0.0130 - val_mae: 0.0874
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0140 - mae: 0.0931 - val_loss: 0.0129 - val_mae: 0.0873
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0133 - mae: 0.0900 - val_loss: 0.0135 - val_mae: 0.0892
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0130 - mae: 0.0881 - val_loss: 0.0126 - val_mae: 0.0868
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0133 - mae: 0.0909 - val_loss: 0.0129 - val_mae: 0.0875
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0125 - mae: 0.0874 - val_loss: 0.0126 - val_mae: 0.0873
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.01

In [33]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0085 - mae: 0.0723
Test Loss: [0.009187637828290462, 0.0746118500828743]


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


a, b y d

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


a, b, c y d